In [1]:
import numpy as np

In [2]:
hbar = 1.05457182e-34 #m^2 kg / s
h = 6.62607015e-34 #J / Hz
c = 299792458 #m / s
u = 1.66053906660e-27 #kg
a0 = 5.29177210903e-11 #m
epsilon0 = 8.8541878128e-12 #F/m

In [3]:
m = 39.96399848*u #kg
alpha = {1064: 597.5, 1550: 381.8, 866: 86.2} #a.u. (see Safronova PRA 73, 022525 (2006))


##def lattice3d(x, y, z, Vx, Vy, Vz, w0_x, w0_y, w0_z, lambda_x=1064, lambda_y=1064, lambda_z=866):
#def lattice3d(x, y, z, Vx, Vy, w0_x, w0_y, lambda_x=1064, lambda_y=1064):
#    kx = 2*np.pi/lambda_x
#    ky = 2*np.pi/lambda_y
#    #kz = 2*np.pi/lambda_z
#
#    V = -Vx * np.exp(-2*(y**2 + z**2)/(w0_x[0]*w0_x[1])) * np.sin(kx*x)**2 \
#        -Vy * np.exp(-2*(x**2 + z**2)/(w0_y[0]*w0_y[1])) * np.sin(ky*y)**2 #\
#        #-Vz * np.exp(-2*(x**2 + y**2)/(w0_z[0]*w0_z[1])) * np.sin(kz*z)**2
#
#    return V
#
#
#
##def second_partial_derivative_x(func, x, y, z, h=1e-10, *args, **kwargs):
##    df_dx2 = (func(x + h, y,z, *args, **kwargs) - 2 * func(x, y,z, *args, **kwargs) + func(x - h, y,z, *args, **kwargs)) / (h**2)    
##    return df_dx2
##
##def second_partial_derivative_y(func, x, y, z, h=1e-10, *args, **kwargs):
##    df_dy2 = (func(x,y + h,z, *args, **kwargs) - 2 * func(x,y,z, *args, **kwargs) + func(x,y - h,z, *args, **kwargs)) / (h**2)    
##    return df_dy2
#
#def second_partial_derivative_z(func, x, y, z, *args, h=1e-10, **kwargs):
#    df_dz2 = (func(x,y,z + h, *args, **kwargs) - 2*func(x,y,z, *args, **kwargs) + func(x,y,z + h, *args, **kwargs)) / (h**2)    
#    return df_dz2


#def second_partial_derivative(x, y, z, Vx, Vy, Vz, x_beam_params, y_beam_params, z_beam_params,):
#    w0_x = x_beam_params['w0']
#    w0_y = y_beam_params['w0']
#    w0_z = z_beam_params['w0']
#    kx = 2*np.pi/(x_beam_params['lambda']*1e-9)
#    ky = 2*np.pi/(y_beam_params['lambda']*1e-9)
#    kz = 2*np.pi/(z_beam_params['lambda']*1e-9)
#
#    V = -2*Vx*kx**2 * np.exp(-(2*(y**2 + z**2))/(w0_x[0]*w0_x[1])) * (np.cos(kx*x)**2 - np.sin(kx*x)**2) \
#        -2*Vy*ky**2 * np.exp(-(2*(x**2 + z**2))/(w0_y[0]*w0_y[1])) * (np.cos(ky*y)**2 - np.sin(ky*y)**2) \
#        -2*Vz*kz**2 * np.exp(-(2*(x**2 + y**2))/(w0_z[0]*w0_z[1])) * (np.cos(kz*z)**2 - np.sin(kz*z)**2)
#    
#    return V    


def get_lattice_depth(beam_params,):
    '''Calculate the lattice depth of lattice alongside direction perpendicular to w0.
    
        Args:
            P: Power per beam [W]
            w0: Beam waist radius in both radial directions [cm x cm]
            lambda_: Wavelength of lattice laser [nm]
            theta: angle between the two interfering lattice beams [rad]

        Returns:
            Er: Recoil energy of lattice [Hz]
            Vlat: Lattice depth [Hz]
    '''
    P = beam_params['P']
    lambda_ = beam_params['lambda']
    w0 = beam_params['w0']
    theta = beam_params['theta']

    a = lambda_*1e-9/(2*np.sin(theta/2)) #m
    Er = h / (8*m*(a**2)) #Hz
    I0 = 2*4*P/(np.pi*(w0[0]*w0[1])) #W/cm^2 (We need the total power of the E-field, which for us is 4 times the beam power because the beam gets retroreflected to form the lattice)

    # look at ipad notes for the derivation of the following formula
    Vlat = alpha[lambda_] * 1e4*(a0**3)/(c*hbar) * I0 #Hz 

    return Er, Vlat



def get_trap_frequencies(Vx, Vy, Vz, x_beam_params, y_beam_params, z_beam_params,):
    '''Calculate trap frequency of 3d lattice in z-direction.

        Args:
            Vx: Lattice depth in x-direction [Hz]
            Vy: Lattice depth in y-direction [Hz]
            w0_x: Beam waist radius in y- and z- direction of x-lattice beam [cm x cm]
            w0_y: Beam waist radius in x- and z- direction of y-lattice beam [cm x cm]
            z: Position of local trap frequency 

        Returns:
            w_trap: Laocal trap frequency [Hz]
    
    '''
    kx = 2*np.pi/(x_beam_params['lambda']*1e-9)
    ky = 2*np.pi/(y_beam_params['lambda']*1e-9)
    kz = 2*np.pi/(z_beam_params['lambda']*1e-9)

    w_trap_x = np.sqrt(2*Vx*h*(kx**2)/m) #Hz
    w_trap_y = np.sqrt(2*Vy*h*(ky**2)/m) #Hz
    w_trap_z = np.sqrt(2*Vz*h*(kz**2)/m) #Hz

    return w_trap_x, w_trap_y, w_trap_z



def get_lamb_dicke_parameter(w_trap, lattice_params):

    lambda_ = lattice_params['lambda']

    k = 2*np.pi/(lambda_*1e-9) #1/m
    w_rec = hbar*k**2/(2*m) #Hz

    eta = np.sqrt(w_rec/w_trap) 

    return eta




<h3>Lattice depth</h3>

In [4]:
x_beam_params = {'P': 10, #W
                 'lambda': 1064, #nm
                 'w0': [350e-4, 175e-4], #cm x cm
                 'theta': np.pi ,#rad
                 }

y_beam_params = {'P': 10, #W
                 'lambda': 1064, #nm
                 'w0': [350e-4, 175e-4], #cm x cm
                 'theta': np.pi ,#rad
                 }

z_beam_params = {'P': 0.5, #W
                 'lambda': 866, #nm
                 'w0': [50e-4, 50e-4], #cm x cm
                 'theta': 16.7/360 * 2*np.pi ,#rad
                 }

Erx, Vx = get_lattice_depth(x_beam_params)
Ery, Vy = get_lattice_depth(y_beam_params)
Erz, Vz = get_lattice_depth(z_beam_params)

print('x direction depth: ')
print('Er = ', Erx*1e-3, 'kHz')
print('Vlat/Er = ', Vx/Erx)
print('\n\n')
print('y direction depth: ')
print('Er = ', Ery*1e-3, 'kHz')
print('Vlat/Er = ', Vy/Ery)
print('\n\n')
print('z direction depth: ')
print('Er = ', Erz*1e-3, 'kHz')
print('Vlat/Er = ', Vz/Erz)

x direction depth: 
Er =  4.409859436263901 kHz
Vlat/Er =  264.030269003267



y direction depth: 
Er =  4.409859436263901 kHz
Vlat/Er =  264.030269003267



z direction depth: 
Er =  0.1403857769827029 kHz
Vlat/Er =  1465.7530268609225


<h3>Trapping frequencies</h3>

In [5]:
w_trap_x, w_trap_y, w_trap_z = get_trap_frequencies(Vx, Vy, Vz, x_beam_params, y_beam_params, z_beam_params,)

print('w_trap_x = 2pi x ', w_trap_x*1e-3 / (2*np.pi), 'kHz')
print('w_trap_y = 2pi x ', w_trap_y*1e-3 / (2*np.pi), 'kHz')
print('w_trap_z = 2pi x ', w_trap_z*1e-3 / (2*np.pi), 'kHz')

eta_x = get_lamb_dicke_parameter(w_trap_x, x_beam_params)
eta_y = get_lamb_dicke_parameter(w_trap_y, y_beam_params)
eta_z = get_lamb_dicke_parameter(w_trap_z, z_beam_params)

print('\n\n')
print('eta_x = ', eta_x)
print('eta_y = ', eta_y)
print('eta_z = ', eta_z)

w_trap_x = 2pi x  143.31168469380702 kHz
w_trap_y = 2pi x  143.31168469380702 kHz
w_trap_z = 2pi x  74.02153037219855 kHz



eta_x =  0.17541695707224672
eta_y =  0.17541695707224672
eta_z =  0.29988661345113604


In [6]:
P = 0.15 #W
w0 = [50e-4, 50e-4] #cm x cm
lambda_ = 866 #nm
theta = 16.7/360 * 2*np.pi #rad


a = lambda_*1e-9/(2*np.sin(theta/2)) #m
Er = h / (8*m*(a**2)) #Hz
V0 = 440*Er

I0 = 2*4*P/(np.pi*(w0[0]*w0[1])) #W/cm^2 

alpha = V0 / (1e4*(a0**3)/(c*hbar) * I0)

print('alpha = ', alpha)

alpha =  86.25373057384968
